In [2]:
import requests
from bs4 import BeautifulSoup
import csv
from fake_useragent import UserAgent
from time import sleep
import numpy as np
import pandas as pd

In [6]:
types = ['自然风光', '山川', '江河湖泊', '地质景观', '海滨海岛', '公园', '峡谷', '瀑布', '地质公园', '文化古迹']

In [79]:
def download_page(url):
    try:
        data = requests.get(url=url,headers=HEADERS,allow_redirect=True).content
        return data
    except:
        pass
    
def download_soup_waitting(url):
    HEADERS = {
    'User-Agent':UserAgent(verify_ssl=False).random,
    'Accept':'application/json, text/javascript, */*; q=0.01',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Cookie':'',
    'Connection':'keep-alive',
    'Pragma':'no-cache',
    'Cache-Control':'no-cache'
}
    try:
        response = requests.get(url=url,headers=HEADERS)
        if response.status_code == 200:
            html = response.content
            html = html.decode('utf-8')
            soup = BeautifulSoup(html,'html.parser')
            return soup
        else:
            sleep(2)
            return download_soup_waitting(url)
    except:
        return ""

In [35]:
url = 'https://piao.qunar.com/ticket/list.htm?keyword=热门景点&region=&from=mpl_search_suggest&subject=自然风光&page=1&sku='
req = requests.get(url=url)
req = BeautifulSoup(req.text)
req = req.find('div',class_='result_list',id='search-list').find_all('div',class_='sight_item')


In [90]:
def getdata(type,url,df):
    req = download_soup_waitting(url)
    sight_items = req.find('div',class_='result_list',id='search-list').find_all('div',class_='sight_item')
    for sight_item in sight_items:
        districts = sight_item['data-districts']
        name = sight_item['data-sight-name']
        id = sight_item['data-id']
        #type = '自然风光'
        level = sight_item.find('span',class_='level')
        level = level.text if level else ''
        product_star_level = sight_item.find('span',class_='product_star_level')
        product_star_level = product_star_level.text if product_star_level else ''
        address = sight_item['data-address']
        intro = sight_item.find('span',class_='intro')
        intro = intro.text if intro else ''
        point = sight_item['data-point']
        hot_num = sight_item.find('span',class_='hot_num').text
        #print(districts,name,id,type,level,product_star_level,address,intro,point)
        df = df.append([[districts,name,id,type,level,product_star_level,address,intro,point,hot_num]],ignore_index=True)
        #df.to_csv('jingdian.csv',mode='a',index=False,header=False)
        #print(df)
        next = req.find('a',class_='next')
        '''if next:
            sleep(1)
            next_url = 'https://piao.qunar.com' + next['href']
            getdata(type,next_url,df)'''
    #print(df)
    df.to_csv('jingdian.csv',mode='a',index=False,header=False)
            

In [91]:
types = ['自然风光', '山川', '江河湖泊', '地质景观', '海滨海岛', '公园', '峡谷', '瀑布', '地质公园', '文化古迹']
for type in [types[1]]:
    df = pd.DataFrame()
    for i in range(1,10):
        url = 'https://piao.qunar.com/ticket/list.htm?keyword=热门景点&region=&from=mpl_search_suggest&subject={type}&page={num}&sku='.format(type=type,num=i)
        print(url)
        getdata(type,url,df)

https://piao.qunar.com/ticket/list.htm?keyword=热门景点&region=&from=mpl_search_suggest&subject=山川&page=1&sku=
https://piao.qunar.com/ticket/list.htm?keyword=热门景点&region=&from=mpl_search_suggest&subject=山川&page=2&sku=
https://piao.qunar.com/ticket/list.htm?keyword=热门景点&region=&from=mpl_search_suggest&subject=山川&page=3&sku=
https://piao.qunar.com/ticket/list.htm?keyword=热门景点&region=&from=mpl_search_suggest&subject=山川&page=4&sku=
https://piao.qunar.com/ticket/list.htm?keyword=热门景点&region=&from=mpl_search_suggest&subject=山川&page=5&sku=
https://piao.qunar.com/ticket/list.htm?keyword=热门景点&region=&from=mpl_search_suggest&subject=山川&page=6&sku=
https://piao.qunar.com/ticket/list.htm?keyword=热门景点&region=&from=mpl_search_suggest&subject=山川&page=7&sku=
https://piao.qunar.com/ticket/list.htm?keyword=热门景点&region=&from=mpl_search_suggest&subject=山川&page=8&sku=
https://piao.qunar.com/ticket/list.htm?keyword=热门景点&region=&from=mpl_search_suggest&subject=山川&page=9&sku=


In [57]:
with open('jingdian.csv','w',encoding='utf-8',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['区域','名称','景点id','类型','级别','热度','地址','特色','经纬度','月销量'])

In [92]:
b = pd.read_csv('jingdian.csv')
b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 10 columns):
区域      150 non-null object
名称      150 non-null object
景点id    150 non-null int64
类型      150 non-null object
级别      120 non-null object
热度      150 non-null object
地址      150 non-null object
特色      0 non-null float64
经纬度     150 non-null object
月销量     150 non-null int64
dtypes: float64(1), int64(2), object(7)
memory usage: 11.8+ KB


In [62]:
url = 'https://piao.qunar.com/ticket/list.htm?keyword=热门景点&region=&from=mpl_search_suggest&subject=自然风光&page=1&sku='
req = requests.get(url=url)
req = BeautifulSoup(req.text)
types = []
for i in req.find('dl',id='subject-list'):
    if i['data-value']:
        types.append(i['data-value'])
print(types)

['自然风光', '山川', '江河湖泊', '地质景观', '海滨海岛', '公园', '峡谷', '瀑布', '地质公园', '文化古迹']


In [15]:
df = pd.DataFrame()

In [19]:
df = df.append([['1','2','3']])

In [23]:
df = df.append([['4','5','6']],ignore_index = True)

In [93]:
import json

# 定义请求url
url = "https://movie.douban.com/j/search_subjects"
# 定义请求头
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36"
}
# 循环构建请求参数并且发送请求
for page_start in range(0, 100, 20):
    params = {
        "type": "movie",
        "tag": "热门",
        "sort": "recommend",
        "page_limit": "20",
        "page_start": page_start
    }
    response = requests.get(
        url=url,
        headers=headers,
        params=params
    )
    # 方式一:直接转换json方法
    # results = response.json()
    # 方式二: 手动转换
    # 获取字节串
    content = response.content
    # 转换成字符串
    string = content.decode('utf-8')
    # 把字符串转成python数据类型
    results = json.loads(string)
    # 解析结果
    for movie in results["subjects"]:
        print(movie["title"], movie["rate"])

续命之徒：绝命毒师电影 8.4
银河补习班 6.2
这个男孩必须被释放 7.5
神奇女侠：血脉 6.1
印度有嘻哈 7.6
送我上青云 7.3
哪吒之魔童降世 8.5
安娜 7.7
寄生虫 8.7
保持沉默 5.4
沉默的证人 5.4
仲夏夜惊魂 6.5
极限逃生 7.6
柳烈的音乐专辑 7.9
小丑回魂2 6.5
大侦探皮卡丘 6.5
深夜食堂 5.1
玩具总动员4 8.7
高草丛中 6.3
铤而走险 6.1
流浪地球 7.9
疯狂的外星人 6.4
烈火英雄 6.6
龙牌之谜 3.6
蜘蛛侠：英雄远征 7.8
速度与激情：特别行动 6.3
恶人传 7.7
使徒行者2：谍影行动 6.4
X战警：黑凤凰 5.9
无名之辈 8.1
飞驰人生 6.9
名侦探柯南：绀青之拳 6.0
疾速备战 7.8
海王 7.6
阿丽塔：战斗天使 7.5
蜘蛛侠：平行宇宙 8.6
爱宠大机密2 7.0
在黑暗中讲述的恐怖故事 6.0
白蛇：缘起 7.9
波西米亚狂想曲 8.7
阿拉丁 7.6
我身体里的那个家伙 7.2
狮子王 7.4
地久天长 7.9
友情以上 6.4
极限职业 7.7
复仇者联盟4：终局之战 8.5
神奇动物：格林德沃之罪 7.1
风中有朵雨做的云 7.2
骡子 7.9
巨鳄风暴 5.8
哆啦A梦：大雄的月球探险记 7.4
大黄蜂 7.0
花椒之味 7.3
无敌破坏王2：大闹互联网 8.1
小委托人 8.0
海市蜃楼 7.7
痛苦与荣耀 8.6
我的一级兄弟 8.3
断裂 6.6
看不见的客人(意大利版) 9.1
密室逃生 7.2
沦落人 8.2
扫毒2天地对决 6.0
黑衣人：全球追缉 5.6
鼠胆英雄 5.4
新喜剧之王 5.7
在无爱之森呐喊 6.8
假面饭店 6.7
你好，之华 6.7
驯龙高手3 7.4
惊奇队长 6.9
老师·好 6.7
昨日青空 6.1
一条狗的使命2 6.9
自卫的艺术 7.4
老千3：独眼杰克 6.2
月影杀痕 6.1
哥斯拉2：怪兽之王 6.3
安娜贝尔3：回家 5.7
来电狂响 5.8
阴曹使者 6.3
“大”人物 6.5
姐姐 6.4
昨日奇迹 6.5
追龙Ⅱ 5.5
我们 6.4
雷霆沙赞！ 6.3
心理测量者SS2：第一卫士 8.0
最好的我们 5.4
丧尸未逝 5.7
小飞象 6.7
胡桃夹子和四个王国 6.0
准备好了没 